# Backward Wrapper

In [12]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
import numpy as np

# import some data to play with
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
Y = (iris.target != 0) * 1


In [13]:
class LogisticRegression:
    
    def fit(self, X, y, lr=0.001, thres=0.001):
        #print("before",X.shape)
        X = np.insert(X, 0, 1, axis=1)
        #print("after",X.shape)
        self.weight = np.zeros(X.shape[1])
        while True:
            prediction = self.sigmoid(np.dot(X, self.weight))
            gradient = np.dot((y - prediction), X)
            self.weight = self.weight + (lr * gradient)
            update = (lr * gradient)
            if np.abs(update).max() < thres:
                break
        
    def predict(self, X):
        X = np.insert(X, 0, 1, axis=1)
        prediction = self.sigmoid(np.dot(X, self.weight))
        return np.vectorize(lambda p: 1 if p >= 0.5 else 0)(prediction)
    
    def sigmoid(self, z):
        return 1/ (1 + np.exp(-1 * z))
    
    
    
    
    def loss_function(self, X, y):
        X = np.insert(X, 0, 1, axis=1)
        prediction = self.sigmoid(np.dot(X, self.weight))
        return np.sum(np.dot(y, (np.log(prediction))) + np.dot((1-y),(np.log(1 - prediction))))

In [14]:
def Cross_validation(X,y):
    kfold=k_fold(X,5)
    losses=[]
    for fold in kfold:
        train_indices,test_indices=fold
        x_train,x_test=X[train_indices],X[test_indices]
        y_train,y_test=y[train_indices],y[test_indices]
        lr=LogisticRegression()
        lr.fit(x_train,y_train)
        loss=lr.loss_function(x_test,y_test)
        losses.append(loss)
        
        
    average=sum(losses)/len(losses)
    return average
    

In [15]:
def k_fold(X,k):
    m=X.shape[0]
    indices=np.random.permutation(m)
    fold_length=m//k
    item_slices=[indices[i * fold_length : (i + 1)* fold_length] for i in range(k)] 
    train_test=[]
    
    for test_index in range(k):
        test=item_slices[test_index]
        train_indices=[i for i in range(k) if test_index!= i]
        train=[]
        for train_index in train_indices:
            train.extend(item_slices[train_index])
            
        train_test.append((train,test))
    return train_test

In [16]:
k=2
Features=[0,1,2,3]
while len(Features) > k:
    averages_cost=[]
    for i in range(len(Features)):
        sub_features=[feature for feature in Features if feature!= Features[i]]
        avg_cost=Cross_validation(X[:,sub_features],Y)
        averages_cost.append(avg_cost)
        
    remove=np.argmin(averages_cost)
    Features.pop(remove)
print(Features)       
    

[0, 1]
